In [1]:
import sys
from pathlib import Path
import numpy as np

current_dir = Path.cwd()
if str(current_dir) not in sys.path:
    sys.path.insert(0, str(current_dir))

from src import (
    DataConfig, ModelingStrategy, BenchmarkPipeline, create_config
)

np.random.seed(42)

In [2]:
data_config = DataConfig(
    features_path="data/processed/train_data_features.feather",
    target_path="data/train_data_target.feather",
    mapping_path="data/feature_mapping_train.pkl"
)

sku_tuples = [(81054, 1334)]
pipeline = BenchmarkPipeline(data_config)

In [3]:
results_xgb_std = pipeline.run_experiment(
    sku_tuples=sku_tuples,
    modeling_strategy=ModelingStrategy.INDIVIDUAL,
    model_type="xgboost_standard",
    hyperparameters={
        "n_estimators": 100,
        "max_depth": 6,
        "learning_rate": 0.3,
        "random_state": 42
    },
    experiment_name="xgb_standard_test"
)

print(f"Trained {results_xgb_std.num_models} XGBoost standard model(s)")

Trained 1 XGBoost standard model(s)


In [4]:
results_xgb_quantile = pipeline.run_experiment(
    sku_tuples=sku_tuples,
    modeling_strategy=ModelingStrategy.INDIVIDUAL,
    model_type="xgboost_quantile",
    hyperparameters={
        "n_estimators": 100,
        "max_depth": 6,
        "learning_rate": 0.3,
        "random_state": 42
    },
    quantile_alphas=[0.5, 0.7, 0.9],
    experiment_name="xgb_quantile_test"
)

print(f"Trained {results_xgb_quantile.num_models} XGBoost quantile model(s)")

Trained 3 XGBoost quantile model(s)


In [8]:
results_lightning_std = pipeline.run_experiment(
    sku_tuples=sku_tuples,
    modeling_strategy=ModelingStrategy.INDIVIDUAL,
    model_type="lightning_standard",
    hyperparameters={
        "hidden_size": 64,
        "learning_rate": 0.0005,
        "num_layers": 2,
        "dropout": 0.2,
        "max_epochs": 50,
        "batch_size": 50,
        "random_state": 42
    },
    experiment_name="lightning_standard_test"
)

print(f"Trained {results_lightning_std.num_models} Lightning standard model(s)")

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name  | Type       | Params | Mode 
---------------------------------------------
0 | model | Sequential | 5.1 K  | train
---------------------------------------------
5.1 K     Trainable params
0         Non-trainable params
5.1 K     Total params
0.020     Total estimated model params size (MB)
8         Modules in train mode
0         Modules in eval mode
/opt/miniconda3/envs/ml_env/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:433: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.
/opt/miniconda3/envs/ml_env/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider 

Trained 1 Lightning standard model(s)


/Users/ivn/Documents/PhD/Transformer Research/Code/Benchmarking/src/evaluation.py:68: RuntimeWarning: invalid value encountered in cast
  y_pred = np.clip(np.round(y_pred).astype(int), 0, None)


In [ ]:
results_lightning_quantile = pipeline.run_experiment(
    sku_tuples=sku_tuples,
    modeling_strategy=ModelingStrategy.INDIVIDUAL,
    model_type="lightning_quantile",
    hyperparameters={
        "hidden_size": 64,
        "learning_rate": 0.001,
        "num_layers": 2,
        "dropout": 0.2,
        "max_epochs": 10,
        "batch_size": 32,
        "random_state": 42
    },
    quantile_alphas=[0.7],
    experiment_name="lightning_quantile_test"
)

print(f"Trained {results_lightning_quantile.num_models} Lightning quantile model(s)")

In [ ]:
results_statquant = pipeline.run_experiment(
    sku_tuples=sku_tuples,
    modeling_strategy=ModelingStrategy.INDIVIDUAL,
    model_type="statquant",
    hyperparameters={
        "method": "interior-point",
        "max_iter": 1000,
        "p_tol": 1e-6,
        "random_state": 42
    },
    quantile_alphas=[0.7],
    experiment_name="statquant_test"
)

print(f"Trained {results_statquant.num_models} statistical quantile model(s)")

In [ ]:
multi_sku_tuples = [(81054, 1334), (80558, 1334)]

results_combined = pipeline.run_experiment(
    sku_tuples=multi_sku_tuples,
    modeling_strategy=ModelingStrategy.COMBINED,
    model_type="xgboost_standard",
    hyperparameters={
        "n_estimators": 50,
        "max_depth": 4,
        "learning_rate": 0.1,
        "random_state": 42
    },
    experiment_name="combined_strategy_test"
)

print(f"Trained {results_combined.num_models} combined model(s) for {len(multi_sku_tuples)} SKUs")

In [ ]:
sample_result = results_xgb_quantile.training_results[0]
print(f"Model type: {sample_result.model_type}")
print(f"Strategy: {sample_result.modeling_strategy.value}")
print(f"SKU tuples: {sample_result.sku_tuples}")
print(f"Quantile level: {sample_result.quantile_level}")
print(f"Training loss: {sample_result.training_loss}")
if sample_result.performance_metrics:
    print(f"RMSE: {sample_result.performance_metrics.get('rmse', 'N/A')}")
    print(f"Coverage: {sample_result.performance_metrics.get('coverage_probability', 'N/A')}")

In [ ]:
for i, result in enumerate(results_lightning_quantile.training_results[:1]):
    rmse = result.performance_metrics.get('rmse', 'N/A')
    rmse_str = f"{rmse:.4f}" if isinstance(rmse, (int, float)) else rmse
    print(f"Model {i+1}: α={result.quantile_level}, RMSE={rmse_str}")